In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from tqdm.notebook import tqdm
import re
import os

In [2]:
trans = str.maketrans('ATGC', 'TACG')

In [3]:
from isotonic_barcode_calling import *
import parallel_process_files

In [4]:
# minimum score (out of 100) on fuzzy protospacer matching
MATCHING_THRESHOLD = 80
IDX_THRESHOLD = 75
# number of workers for distance computation in calling
NUM_WORKERS = 8

In [5]:
DIRECTORY = "/data1/normantm/angel/sequencing/AP1_816_puro_round2_1_output_NoFilter/Samples/DefaultProject"

In [6]:
def get_folder_names(directory):
    # List all entries in the directory
    entries = os.listdir(directory)
    
    # Filter out only folders
    folders = sorted([entry for entry in entries if os.path.isdir(os.path.join(directory, entry))])
    
    return folders

SAMPLE_NAMES = get_folder_names(DIRECTORY)

In [7]:
barcodes = pd.read_csv("120k_barcodes.csv", header=None)[1].str[30:55].values

In [8]:
K562_separate_sgRNA1 = pd.read_csv("twist_K562_816_separate_sgRNA1.csv", header=None, index_col=0)
K562_separate_sgRNA2 = pd.read_csv("twist_K562_816_separate_sgRNA2.csv", header=None, index_col=0)

K562_separate_sgRNA1["sgRNA1"] = K562_separate_sgRNA1[1].str[34:54].str.translate(trans).str[::-1]
K562_separate_sgRNA2["sgRNA2"] = K562_separate_sgRNA2[1].str[70:90]

In [9]:
i1_map = pd.Series({
    "1_1": "CGTACTAG",
    "1_2": "AGGCAGAA", 
    "1_3": "TCCTGAGC",
    "2_1": "GGACTCCT",
    "2_2": "TAGGCATG",
    "2_3": "CTCTCTAC",
    "3_1": "AAGAGGCA",
    "3_2": "GTAGAGGA",
    "3_3": "GCTCATGA"
    })
i2_map = pd.Series({"identity": "TAGATCGC", "reporter": "CTCTCTAT"})

In [10]:
def process_sample(sample, separate_sgRNA1, separate_sgRNA2, i1_map, i2_map, run_name, output_dir):
    print("reading UMI")
    umi = parallel_process_files.fast_fastq_sequences(f"{DIRECTORY}/{sample}/{sample}_UMI_I1.fastq.gz", pigz_threads=4)
    print("reading guide 2")
    g2 = parallel_process_files.parallel_extract_protospacer2(f"{DIRECTORY}/{sample}/{sample}_R2.fastq.gz",
                                                              workers=12, chunksize=1000000, pigz_threads=4)
    print("reading barcode")
    b = parallel_process_files.parallel_barcode_extractor(f"{DIRECTORY}/{sample}/{sample}_R2.fastq.gz",
                                                          workers=12, chunksize=1000000, pigz_threads=4)
    print("reading guide 1")
    g1 = parallel_process_files.parallel_protospacer_extractor(f"{DIRECTORY}/{sample}/{sample}_R1.fastq.gz", 
                                                               five_bp="TAAAC", workers=12, chunksize=1000000,
                                                               pigz_threads=4)
    print("reading index 1")
    i1 = parallel_process_files.fast_fastq_sequences(f"{DIRECTORY}/{sample}/{sample}_I1.fastq.gz", pigz_threads=4)
    print("reading index 2")
    i2 = parallel_process_files.fast_fastq_sequences(f"{DIRECTORY}/{sample}/{sample}_I2.fastq.gz", pigz_threads=4)
    
    unique_b = pd.unique(b)
    unique_i1 = pd.unique(i1)
    unique_i2 = pd.unique(i2)

    print("mapping barcodes")
    with open(f"/data1/normantm/angel/freebarcodes/{run_name}_unique_b_{sample}.fastq", "w") as f:
        for i, ba in enumerate(unique_b):
            if "N" in ba:
                continue
            f.write(f"@{i}\n")
            f.write(f"{ba}\n")
            f.write("+\n")
            Q = ['I']*25
            Q = "".join(Q)
            f.write(f"{Q}\n")
            
    os.system("freebarcodes decode /data1/normantm/angel/freebarcodes/120k_barcodes15-2.txt,/data1/normantm/angel/freebarcodes/120k_barcodes10-1.txt " +\
              f"/data1/normantm/angel/freebarcodes/{run_name}_unique_b_{sample}.fastq --output-dir /data1/normantm/angel/freebarcodes/")
    
    decoded_unique_b = pd.read_csv(f"/data1/normantm/angel/freebarcodes/{run_name}_unique_b_{sample}_decoded.txt", sep="\t", header=None, index_col=0)
    decoded_unique_b.columns = ["b15", "b10", "seq"]
    decoded_unique_b["barcode_mapped"] = decoded_unique_b["b15"] + decoded_unique_b["b10"]
    decoded_unique_b.drop_duplicates(inplace=True)
    decoded_unique_b.query("barcode_mapped in @barcodes", inplace=True)
    decoded_unique_b.set_index("seq", inplace=True)

    print("aligning protospacers")
    idx_b = pd.Index(b).isin(decoded_unique_b.index)

    filtered_g1 = g1[idx_b]
    filtered_g2 = g2[idx_b]
    unique_p = pd.unique(np.concatenate([pd.unique(filtered_g1), pd.unique(filtered_g2)]))

    alignments_p = chunked_call_alignments_new(unique_p, separate_sgRNA1.sgRNA1, num_chunks=int(len(unique_p)/1000), num_workers=NUM_WORKERS)
    alignments_p.to_csv(f"{output_dir}/guide_alignments.csv")
    alignments_p.query("score >= @MATCHING_THRESHOLD", inplace=True)

    print("mapping indices")
    alignments_i1 = chunked_call_alignments_new(unique_i1, i1_map, num_chunks=int(len(unique_i1)/1000), num_workers=NUM_WORKERS)
    alignments_i2 = chunked_call_alignments_new(unique_i2, i2_map, num_chunks=int(len(unique_i2)/1000), num_workers=NUM_WORKERS)
    alignments_i1.query("score >= @IDX_THRESHOLD", inplace=True)
    alignments_i2.query("score >= @IDX_THRESHOLD", inplace=True)

    print("filtering reads")
    idx_p1 = pd.Index(g1).isin(alignments_p.index)
    idx_p2 = pd.Index(g2).isin(alignments_p.index)
    idx_i1 = pd.Index(i1).isin(alignments_i1.index)
    idx_i2 = pd.Index(i2).isin(alignments_i2.index)
    idx_umi = np.char.find(umi, "N") == -1
    
    filtered_idx = idx_p1 & idx_p2 & idx_b & idx_i1 & idx_i2 & idx_umi
    
    g1, g2 = g1[filtered_idx], g2[filtered_idx]
    b = b[filtered_idx]
    umi = umi[filtered_idx]
    i1, i2 = i1[filtered_idx], i2[filtered_idx]

    print("writing to files")
    with open(f"{output_dir}/processed_filtered_reads_{sample}.csv", "a") as f:
        f.write("p1_identity,p2_identity,p1_score,p2_score,sample,transcript,barcode_mapped,UMI\n")
    
        size = 2000000
        chunks = np.arange(0, len(g1), size)
        
        for i in chunks:
                
            lines = np.array([alignments_p.loc[g1[i:i+size]].identity.values,
                              alignments_p.loc[g2[i:i+size]].identity.values,
                              alignments_p.loc[g1[i:i+size]].score.values.astype(int).astype(str),
                              alignments_p.loc[g2[i:i+size]].score.values.astype(int).astype(str),
                              alignments_i1.loc[i1[i:i+size]].identity.values,
                              alignments_i2.loc[i2[i:i+size]].identity.values,
                              decoded_unique_b.loc[b[i:i+size]].barcode_mapped.values,
                              umi[i:i+size]])
                
            lines = [",".join(a)+"\n" for a in lines.T]
            
            f.writelines(lines)

In [11]:
output_dir = "/data1/normantm/angel/sequencing/AP1_816_puro_round2_1_output_NoFilter"
run_name = "K562_AP1_816_puro_round2_1"

In [ ]:
for sample in SAMPLE_NAMES:
   process_sample(sample, K562_separate_sgRNA1, K562_separate_sgRNA2, i1_map, i2_map, run_name, output_dir)